In [1]:
# import libraries 
import pandas as pd
import numpy as np
import subprocess

In [4]:
#read the file as tsv  
#old ver:"/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_merged_ver_2_with_DNA_RNA_seq_and_codon_19_03_23.tsv"
# "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_merged_ver_2_with_DNA_RNA_seq_and_codon_22_03_23_after_codon_fix.tsv"
final_table_file = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_RNA_seq.tsv"
final_table_df = pd.read_csv(final_table_file, sep='\t')
# Displaying a table with all its columns open
pd.set_option('display.max_columns', None)
final_table_df

/tmp/ipykernel_32080/2384710499.py:5: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  final_table_df = pd.read_csv(final_table_file, sep='\t')


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG,Gln,Gln
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 

BLAT on DNA 

In [5]:
def create_fasta_file_for_all_seq(kind_of_seq,path_to_write):
    # create list of fasta format to all sequence
    fasta_list = [">" + str(x) +"\n" +str(y)+"\n" for x, y in final_table_df[kind_of_seq].iteritems()]
    # open file and save all fasta seq 
    ofile = open(path_to_write, "w")
    for i in range(len(fasta_list)):
        ofile.write(fasta_list[i])
    ofile.close()
# sum all hits 
def sum_amount_of_hits_for_each_query_and_assign_results(blat_results_df,original_df,col_name_min_identity):
    # create dataframe that sum how much hits there is for every query
    amount_of_hits_df = blat_results_df.groupby(['Query_id']).size().reset_index(name='Amount')
    #convert the Query_id column to list
    amount_list = amount_of_hits_df['Query_id'].to_list()
    #Initialize the entire column in the dataframe with the value Nan
   
    original_df[col_name_min_identity] = np.nan
    # loop that run on the query id in the amount df and add the sum of hits to the original df 
    for ix in amount_list:
        original_df.loc[ix,col_name_min_identity] = amount_of_hits_df.loc[amount_of_hits_df['Query_id'] == ix, 'Amount'].values
# read blat results and return df
def read_blat_results(blat_file,min_alignment_length,min_identity_precent):
    col_names = ["Query_id","Subject_id","identity_precent","alignment_length","mismatches","gap_openings","q_start","q_end","s_start","s_end","e-value","bit_score"]
    blat_results = pd.read_table(blat_file,names=col_names)
    # filter results that the alignment length is shorter than 21
    blat_results = blat_results[blat_results['alignment_length']>min_alignment_length]
    # filter results that the minimal identity is lower than
    blat_results = blat_results[blat_results['identity_precent']>=min_identity_precent]
    return blat_results



In [6]:
# A list of the percentage of identity we want to get in the results of the ballot
list_of_percent=[85,90,95,100]#[85]#,90,95,100]#[85,90,95,100]
for percent in list_of_percent:
    #minimal_identity = '-minIdentity=' + str(percent)
    create_fasta_file_for_all_seq("dna_seq","/private5/Projects/dadush/clinvar_base_editing/dna_seq_adar.fa")
    # run blat on dna
    blat_path = '/home/alu/gabayo2/tools/blat/blat'
    genome_refernce = '/private/dropbox/Genomes/Human/hg38/hg38.fa'
    queries_path = '/private5/Projects/dadush/clinvar_base_editing/dna_seq_adar.fa'
    results_path = '/private5/Projects/dadush/clinvar_base_editing/blat_result_dna_seq_adar.psl'
    #subprocess.run([blat_path, genome_refernce ,queries_path,'-out=blast8',minimal_identity,results_path])
    # running BLAT
    subprocess.run([blat_path, genome_refernce ,queries_path,'-out=blast8',results_path])

    blat_results_df =read_blat_results('/private5/Projects/dadush/clinvar_base_editing/blat_result_dna_seq_adar.psl',20,percent)
    col_results_name = 'DNA off-target hits -minIdentity=' + str(percent)
    sum_amount_of_hits_for_each_query_and_assign_results(blat_results_df,final_table_df,col_results_name)
    #final_table_without_nan_results_df =final_table_df[~final_table_df[col_results_name].isnull()]
   
    #  #convert the column DNA off-target hits from float to intger
    # final_table_without_nan_results_df[col_results_name] = final_table_without_nan_results_df[col_results_name].astype('int64')
    # # Subtract 1 from the amount of hits to lower the hits that are actually against the given sequence
    # final_table_without_nan_results_df[col_results_name]= final_table_without_nan_results_df[col_results_name]-1

#final_table_without_nan_results_df
final_table_df

Loaded 3209286105 letters in 455 sequences
Searched 4039033 bases in 98513 sequences
Loaded 3209286105 letters in 455 sequences
Searched 4039033 bases in 98513 sequences
Loaded 3209286105 letters in 455 sequences
Searched 4039033 bases in 98513 sequences
Loaded 3209286105 letters in 455 sequences
Searched 4039033 bases in 98513 sequences


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg,1.0,1.0,1.0,1.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala,1.0,1.0,1.0,1.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG,Gln,Gln,1.0,1.0,1.0,1.0
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretat

In [7]:

for percent in list_of_percent:
    # create column name
    col_results_name = 'DNA off-target hits -minIdentity=' + str(percent)
    final_table_df[col_results_name] = final_table_df[col_results_name].fillna(0)
    #convert the column DNA off-target hits from float to intger
    final_table_df[col_results_name] = final_table_df[col_results_name].astype('int64')
    # Subtract 1 from the amount of hits to lower the hits that are actually against the given sequence
    final_table_df[col_results_name]= final_table_df[col_results_name]-1
final_table_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg,0,0,0,0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala,0,0,0,0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG,Gln,Gln,0,0,0,0
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9

In [12]:
# Checking the distribution of the results Any result that is -1 indicates an error and that BLAT did not find even one sequence similar to it
final_table_df.groupby('DNA off-target hits -minIdentity=100')['DNA off-target hits -minIdentity=100'].count()

DNA off-target hits -minIdentity=100
-1        307
 0      92952
 1       3563
 2        513
 3        273
 4        180
 5        161
 6        125
 7        116
 8         77
 9        129
 10        72
 11        12
 12         7
 13         4
 15         3
 16         5
 18         2
 19         4
 23         1
 26         2
 32         1
 38         3
 768        1
Name: DNA off-target hits -minIdentity=100, dtype: int64

In [6]:
# Find cases where the off target is not equal
#final_table_df[(final_table_df['DNA off-target hits -minIdentity=95']!=final_table_df['DNA off-target hits -minIdentity=100'])&(final_table_df['DNA off-target hits -minIdentity=100']!=-1)&(final_table_df['DNA off-target hits -minIdentity=95']!=-1)]

BLAT on RNA 

In [7]:
# Creating a column with the name of the gene so that we can identify its isoforms
final_table_df['Gene'] = final_table_df['geneId'].str.split('|').str[1]
final_table_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,CGA,Arg,Arg,0,SAMD11
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,GCG,Ala,Ala,0,KLHL17
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,0,ISG15
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,0.0,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCC

In [8]:
for percent in list_of_percent:
    minimal_identity = '-minIdentity=' + str(percent)
    col_results_name = 'RNA off-target hits ' + minimal_identity
    #create fasta file for RNA sequences
    create_fasta_file_for_all_seq("new_exon_seq","/private5/Projects/dadush/clinvar_base_editing/rna_seq_adar.fa")
    # BLAT command 
    blat_path = '/home/alu/gabayo2/tools/blat/blat'
    genome_refernce = "/private5/Projects/dadush/clinvar_base_editing/my_rna_utr_seq_fasta_file.fa"
    queries_path = '/private5/Projects/dadush/clinvar_base_editing/rna_seq_adar.fa'
    results_path = '/private5/Projects/dadush/clinvar_base_editing/blat_result_rna_seq_adar.psl'

    subprocess.run([blat_path, genome_refernce ,queries_path,'-out=blast8',results_path]) #'-q=rna',
    blat_results =read_blat_results('/private5/Projects/dadush/clinvar_base_editing/blat_result_rna_seq_adar.psl',20,percent)
    blat_results['Subject_id'] = blat_results['Subject_id'].str.replace('hg38_ncbiRefSeqCurated_','')
    amount_of_hits_adar_df = blat_results.groupby(['Query_id']).size().reset_index(name='Amount')


    # clean the Subject_id col because there are refesq with the same name because it is in two chromosomes. So at first I connected the chromosome name to the refseq name so we could run BLAT(because blat can't run if there are sequences with same refseq id) and now I'm just leaving the refseq name 
    blat_results['Subject_id'] = blat_results['Subject_id'].str.split("-").str[0]

    #other part
    rna_gene_file = "/private5/Projects/dadush/clinvar_base_editing/blat_running/refseq_name_and_thier_gene"
    col_names = ['refseq_name','gene_id']
    rna_gene_df = pd.read_csv(rna_gene_file, sep='\t',names=col_names,skiprows=1)
    # loop that run on the originl df
    for index, row in final_table_df.iterrows():
        # get data frame from blat results of specific query id 
        blat_check_df =blat_results[blat_results["Query_id"].isin([index])]
        #print(blat_check_df)
    #if there is not result for this index assign nan value
        if blat_check_df.empty:
        #return np.nan
            final_table_df.loc[index,col_results_name] = np.nan
    #if there is result of blat for this query id 
        else:
            #print(blat_check_df)
            # get the gene name of query 
            gene_name = row['Gene']
            #print(gene_name)
            # get list of resfeq id that relate to same gene name so we get all isoforms
            family_rna_list = rna_gene_df.loc[rna_gene_df["gene_id"]==gene_name,'refseq_name'].values.tolist()
            # from blat result of specific query id drop all rows that contain refseq id from list because it is isoform
            blat_check_df = blat_check_df[~blat_check_df['Subject_id'].isin(family_rna_list)]
            # assign the number of hits after we have reduced the amount of isoforms from them
            final_table_df.loc[index,col_results_name] = len(blat_check_df)
final_table_df

Loaded 309196101 letters in 90888 sequences
Searched 3269769 bases in 98513 sequences


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,CGA,Arg,Arg,0,SAMD11,0.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,GCG,Ala,Ala,0,KLHL17,0.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,0,ISG15,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,0.0,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCC

In [9]:
# save df
final_table_df.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_RNA_seq_and_off_target.tsv", index=False,sep="\t")

In [2]:
final_table_file = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_RNA_seq_and_off_target.tsv"
final_table_df = pd.read_csv(final_table_file, sep='\t')
# Displaying a table with all its columns open
pd.set_option('display.max_columns', None)
final_table_df

/tmp/ipykernel_38740/4186644955.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  final_table_df = pd.read_csv(final_table_file, sep='\t')


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg,0,0,0,0,SAMD11,0.0,0.0,0.0,0.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala,0,0,0,0,KLHL17,0.0,0.0,0.0,0.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,ISG15,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGC

In [8]:
final_table_df.groupby('RNA off-target hits -minIdentity=100')['RNA off-target hits -minIdentity=100'].count()
final_table_df[(final_table_df['RNA off-target hits -minIdentity=85'].isna())&(~final_table_df['new_exon_seq'].isna())]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100
21,chr1,1232278,1232279,A>G,1,+,1232278,1232279,"210,0,0",1,1,0,60484|NM_080605.4(B3GALT6):c.1A>G (p.Met1Val),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,initiatior codon variant,786200938.0,nAn,RCV000054390|RCV001560169,N,Spondyloepimetaphyseal dysplasia with joint la...,"MONDO:MONDO:0010075, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Met1Val""], [""c.1A>G"", ""-...",nAn,2,"Sep 13,2019",nAn,"ClinGen:CA144521, <a target=_blank href='http:...","NM_080605.4(B3GALT6):c.1A>G (p.Met1Val), Type:...",P,GERM,SUBST,1,1,60484,nAn,1,1232279.0,60484.0,A,G,.,.,"ALLELEID=75079;CLNDISDB=MONDO:MONDO:0010075,Me...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_080605.4,GCCGGCCCGGCGCGGGCGCCATGAAGCTGCTGCGGCGGGCG,1,ATGAAGCTGCTGCGGCGGGCG,0.0,ATG,Met,Met,0,0,0,0,B3GALT6,NaN,NaN,NaN,NaN
22,chr1,1232279,1232280,T>A,1,+,1232279,1232280,"210,0,0",1,1,0,1067539|NM_080605.4(B3GALT6):c.2T>A (p.Met1Lys),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,initiatior codon variant,nAn,nAn,RCV001378837,N,Spondyloepimetaphyseal dysplasia with joint la...,"MONDO:MONDO:0019675, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1232280T>A"", ""-""], [""g.5032T>A"", ""-""], [""...",nAn,1,"Aug 28,2020",nAn,nAn,"NM_080605.4(B3GALT6):c.2T>A (p.Met1Lys), Type:...",LP,GERM,SUBST,1,1,1067539,nAn,1,1232280.0,1067539.0,T,A,.,.,"ALLELEID=1054735;CLNDISDB=MONDO:MONDO:0014139,...",chr1,Likely_pathogenic,nAn,nAn,A,G,.,NM_080605.4,CCGGCCCGGCGCGGGCGCCATGAAGCTGCTGCGGCGGGCGT,2,ATGAAGCTGCTGCGGCGGGCGT,1.0,ATG,Met,Met,0,0,0,0,B3GALT6,NaN,NaN,NaN,NaN
129,chr1,2412500,2412501,T>C,1,-,2412500,2412501,"210,0,0",1,1,0,162434|NM_002617.4(PEX10):c.2T>C (p.Met1Thr),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,5192|PEX10,initiatior codon variant,724160002.0,nAn,RCV000149812|RCV000665679,N,Peroxisome biogenesis disorder 6B|Peroxisome b...,"MONDO:MONDO:0013937, <a target=_blank href='ht...","germline,unknown",GRCh38,1p36.32,"[[""c.2T>C"", ""p.Met1Thr""], [""g.5071T>C"", ""-""], ...",nAn,2,"Feb 24,2017",nAn,"ClinGen:CA175040, <a target=_blank href='http:...","NM_002617.4(PEX10):c.2T>C (p.Met1Thr), Type: s...",P,GERM,SUBST,1,1,162434,nAn,1,2412501.0,162434.0,A,G,.,.,"ALLELEID=172125;CLNDISDB=MONDO:MONDO:0013936,M...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_002617.4,ACCACCCGAACCCGCGGCCATGGCCCCGGCCGCCGCCAGCC,2,ATGGCCCCGGCCGCCGCCAGCC,1.0,ATG,Met,Met,0,0,0,0,PEX10,NaN,NaN,NaN,NaN
130,chr1,2412501,2412502,A>G,2,-,2412501,2412502,"210,0,0",1,1,0,280002|NM_002617.4(PEX10):c.1A>G (p.Met1Val),Pathogenic/Likely pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,5192|PEX10,initiatior codon variant,886041314.0,nAn,RCV000341916|RCV000666568|RCV001385998,N,not provided|Peroxisome biogenesis disorder 6A...,"MedGen:CN517202|MONDO:MONDO:0013936, <a target...","germline,unknown",GRCh38,1p36.32,"[[""c.1A>G"", ""p.Met1Val""], [""g.5070A>G"", ""-""], ...",nAn,3,"Sep 02,2020",nAn,ClinGen:CA106027

In [9]:
# Because of a bug that exists in pandas with a column named 'name', I changed its name
final_table_df = final_table_df.rename({'name':'mutation'}, axis=1)
final_table_df

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg,0,0,0,0,SAMD11,0.0,0.0,0.0,0.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala,0,0,0,0,KLHL17,0.0,0.0,0.0,0.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,ISG15,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTG

In [10]:
# A function that create value of the ADAR motif
def get_value_of_adar_motif(change, before,after):
    list_of_nucleotide = ['A','T','C','G']
    if change=='G>A':
        # remove 'G' from list because 'G' before the edit site is not ADAR motif
        list_of_nucleotide.remove('G')
        if before in list_of_nucleotide:
            return 'yes'
        else:
            return 'no'
    elif change == 'C>T':
        # remove 'C' from list because 'C' after the edit site means there is a G before the editing site on the opposite strand, so it is not ADAR motif
        list_of_nucleotide.remove('C')
        if after in list_of_nucleotide:
            return 'yes'
        else:
            return 'no'
    else:
        return np.nan

In [11]:
# get adar motif
final_table_df['ADAR motif'] = final_table_df.apply(lambda x: get_value_of_adar_motif(x.mutation, x.base_before,x.base_after),axis=1)
final_table_df

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100,ADAR motif
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg,0,0,0,0,SAMD11,0.0,0.0,0.0,0.0,yes
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala,0,0,0,0,KLHL17,0.0,0.0,0.0,0.0,NaN
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,ISG15,NaN,NaN,NaN,NaN,yes
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCT

In [13]:
# A visual check on the UCSC website that indeed the results we received are correct
final_table_df[(final_table_df['mutation']=='G>A')&(final_table_df['ADAR motif']=='yes')]

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100,ADAR motif
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,ISG15,NaN,NaN,NaN,NaN,yes
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn,1,1014228.0,402986.0,G,A,.,.,AF_ESP=0.40158;AF_EXAC=0.37025;AF_TGP=0.33886;...,chr1,Conflicting_interpretations_of_pathogenicity,"Benign(3),Pathogenic(1)",yes,A,C,.,NM_005101.4,CAAATGCGACGAACCTCTGAGCATCCTGGTGAGGAATAACA,248,CAAATGCGACGAACCTCTGAGCATCCTGGTGAGGAATAACA,1.0,AGC,Ser,Ser,0,0,0,0,ISG15,0.0,0.0,0.0,0.0,yes
15,chr1,1050448,1050449,G>A,0,+,1050448,1050449,"210,0,0",1,1,0,1284257|NM_198576.4(AGRN):c.4999G>A (p.Val1667...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,nAn,nAn,RCV001698934,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""g.1050449G>A"", ""-""], [""g.35327G...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.4999G>A (p.Val1667Met), Ty...",P,GERM,SUBST,1,0,1284257,nAn,1,1050449.0,1284257.0,G,A,.,.,"ALLELEID=1274112;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_198576.4,AGAAGATGGCGCTGGAGGTCGTGTTCCTGGCACGAGGCCCC,4999,AGAAGATGGCGCTGGAGGTCGTGTTCCTGGCACGAGGCCCC,0.0,GTG,Val,Val,0,0,0,0,AGRN,0.0,0.0,0.0,0.0,yes
16,chr1,1050472,1050473,G>A,0,+,1050472,1050473,"210,0,0",1,1,0,243039|NM_198576.4(AGRN):c.5023G>A (p.Gly1675Ser),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,764160563.0,nAn,RCV000235025,N,Congenital myasthenic syndrome,"MONDO:MONDO:0018940, MeSH:D020294, <a target=_...",germline,GRCh38,1p36.33,"[[""c.5023G>A"", ""p.Gly1675Ser""], [""g.35351G>A"",...",nAn,1,"Jul 14,2016",nAn,ClinGen:CA509824,"NM_198576.4(AGRN):c.5023G>A (p.Gly1675Ser), Ty...",P,GERM,SUBST,1,0,243039,nAn,1,105047

In [14]:
# A visual check on the UCSC website that indeed the results we received are correct
final_table_df[(final_table_df['mutation']=='G>A')&(final_table_df['ADAR motif']=='no')]

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,DNA off-target hits -minIdentity=90,DNA off-target hits -minIdentity=95,DNA off-target hits -minIdentity=100,Gene,RNA off-target hits -minIdentity=85,RNA off-target hits -minIdentity=90,RNA off-target hits -minIdentity=95,RNA off-target hits -minIdentity=100,ADAR motif
6,chr1,1022224,1022225,G>A,1,+,1022224,1022225,"210,0,0",1,1,0,243036|NM_198576.4(AGRN):c.226G>A (p.Gly76Ser),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,756623659.0,nAn,RCV000235037|RCV001377978,N,Congenital myasthenic syndrome|Myasthenic synd...,"MONDO:MONDO:0018940, MeSH:D020294, <a target=_...",germline,GRCh38,1p36.33,"[[""c.226G>A"", ""p.Gly76Ser""], [""g.7103G>A"", ""-""...",nAn,2,"Sep 09,2020",nAn,ClinGen:CA507784,"NM_198576.4(AGRN):c.226G>A (p.Gly76Ser), Type:...",LP,GERM,SUBST,1,1,243036,nAn,1,1022225.0,243036.0,G,A,.,.,AF_EXAC=0.00001;ALLELEID=244110;CLNDISDB=MONDO...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198576.4,GGGTCTGGCGGTACTTGAAGGGCAAAGACCTGGTGGCCCGG,226,GGGTCTGGCGGTACTTGAAGGGCAAAGACCTGGTGGCCCGG,0.0,GGC,Gly,Gly,0,0,0,0,AGRN,0.0,0.0,0.0,0.0,no
27,chr1,1232743,1232744,G>A,0,+,1232743,1232744,"210,0,0",1,1,0,60486|NM_080605.4(B3GALT6):c.466G>A (p.Asp156Asn),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,missense variant,397514718.0,nAn,RCV000054392,N,Spondyloepimetaphyseal dysplasia with joint la...,"MONDO:MONDO:0010075, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Asp156Asn""], [""-"", ""p.As...",nAn,1,"Jun 06,2013",nAn,"ClinGen:CA144524, UniProtKB:Q96L58#VAR_070136,...","NM_080605.4(B3GALT6):c.466G>A (p.Asp156Asn), T...",P,GERM,SUBST,1,0,60486,nAn,1,1232744.0,60486.0,G,A,.,.,"ALLELEID=75081;CLNDISDB=MONDO:MONDO:0010075,Me...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_080605.4,TCGAGTTCGTGCTCAAGGCGGACGACGACTCCTTCGCGCGG,466,TCGAGTTCGTGCTCAAGGCGGACGACGACTCCTTCGCGCGG,0.0,GAC,Asp,Asp,0,0,0,0,B3GALT6,0.0,0.0,0.0,0.0,no
58,chr1,1787377,1787378,G>A,0,-,1787377,1787378,"210,0,0",1,1,0,224718|NM_002074.5(GNB1):c.976G>A (p.Ala326Thr),Pathogenic/Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2782|GNB1,missense variant,869312826.0,nAn,RCV000210260|RCV000755057,N,"Global developmental delay,Seizures,Generalize...","Human Phenotype Ontology:HP:0000754, Human Phe...","de novo,unknown",GRCh38,1p36.33,"[[""c.976G>A"", ""-""], [""g.1787378C>T"", ""-""], [""g...",nAn,2,"May 05,2016",nAn,"ClinGen:CA353569, UniProtKB:P62873#VAR_076651","NM_002074.5(GNB1):c.976G>A (p.Ala326Thr), Type...",LP,GERM,SUBST,1,0,224718,nAn,1,1787378.0,224718.0,C,T,.,.,ALLELEID=226495;CLNDISDB=.|.|Human_Phenotype_O...,chr1,Pathogenic/Likely_pathogenic,nAn,nAn,G,C,.,NM_002074.5,GCGTGACTGACGATGGCATGGCTGTGGCGACAGGGTCCTGG,976,GCGTGACTGACGATGGCATGGCTGTGGCGACAGGGTCCTGG,0.0,GCT,Ala,Ala,0,0,0,0,GNB1,0.0,0.0,0.0,0.0,no
70,chr1,1804501,1804502,G>A,1,-,1804501,1804502,"210,0,0",1,1,0,996797|NM_002074.5(GNB1):c.347G>A (p.Gly116Asp),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2782|GNB1,missense variant,nAn,nAn,RCV001291588,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1804502C>T"", ""-""], [""g.91616G>A"", ""-""], [...",nAn,1,"Feb 01,2021",nAn,nAn,"NM_

In [15]:
# save df
final_table_df.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_RNA_seq_and_off_target_and_motif.tsv", index=False,sep="\t")

add codon to stop lost mutation

In [16]:
# #read the file as tsv  
# final_table_file = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_merged_ver_2_with_DNA_RNA_seq_and_codon_and_off_target_and_motif_22_03_23_after_codon_fix.tsv"
# final_table_df = pd.read_csv(final_table_file, sep='\t')
# pd.set_option('display.max_columns', None)
# final_table_df

/tmp/ipykernel_3047/3994958214.py:3: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  final_table_df = pd.read_csv(final_table_file, sep='\t')


,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,CGA,Arg,Arg,0,SAMD11,0.0,yes
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,GCG,Ala,Ala,0,KLHL17,0.0,NaN
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,0,ISG15,NaN,yes
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,0.0,ACCCGAG

In [17]:
# #read the file as tsv  
# stop_lost_file = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/stop_lost_only_143_rows_with_codon.bed"
# stop_lost_df = pd.read_csv(stop_lost_file, sep='\t')
# pd.set_option('display.max_columns', None)
# stop_lost_df=stop_lost_df.rename({'nAn':'codon'},axis=1)
# stop_lost_df

,chrom,chromStart_gtf,chromEnd_gtf,transcript_id,ID,strand_gtf,codon
0,chr1,9982699,9982701,NM_022787.4,37132,+,TAG
1,chr1,11790680,11790682,NM_005957.5,187906,-,TGA
2,chr1,47417273,47417275,NM_012186.3,1077110,+,TGA
3,chr1,47417273,47417275,NM_012186.3,1077109,+,TGA
4,chr1,47417273,47417275,NM_012186.3,30157,+,TGA
...,...,...,...,...,...,...,...
138,chrX,154030367,154030369,NM_001110792.2,143490,-,TGA
139,chrX,154030367,154030369,NM_001110792.2,834679,-,TGA
140,chrX,154564459,154564461,NM_001099857.5,418983,+,TAG
141,chrX,154564459,154564461,NM_001099857.5,11450,+,TAG


In [18]:
# # left join between the final df and the results of bedtools get fasta og stop lost mutation
# new_final_df = pd.merge(final_table_df,stop_lost_df,left_on=['chrom','ID'],right_on=['chrom','ID'], how='left').drop(['chromStart_gtf_y','chromEnd_gtf_y','transcript_id_y','strand_gtf_y'],axis=1)
# new_final_df


,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf_x,chromEnd_gtf_x,strand_gtf_x,loc_in_frame,transcript_id_x,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif,codon
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,CGA,Arg,Arg,0,SAMD11,0.0,yes,NaN
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,GCG,Ala,Ala,0,KLHL17,0.0,NaN,NaN
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,0,ISG15,NaN,yes,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101

In [19]:
# # Choosing the actual value from the two columns that contain the codon
# new_final_df['codon_seq'] = new_final_df.filter(like='codon').ffill(axis=1).iloc[:,-1]
# new_final_df

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf_x,chromEnd_gtf_x,strand_gtf_x,loc_in_frame,transcript_id_x,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif,codon,codon_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,CGA,Arg,Arg,0,SAMD11,0.0,yes,NaN,CGA
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,GCG,Ala,Ala,0,KLHL17,0.0,NaN,NaN,GCG
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,0,ISG15,NaN,yes,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,101

In [20]:
# # Checking that it did work
# new_final_df[new_final_df['molConseq']=='stop lost'].head(n=50)

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf_x,chromEnd_gtf_x,strand_gtf_x,loc_in_frame,transcript_id_x,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif,codon,codon_seq
324,chr1,9982698,9982699,T>C,0,+,9982698,9982699,"210,0,0",1,1,0,37132|NM_022787.4(NMNAT1):c.838T>C (p.Ter280Gln),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,64802|NMNAT1,stop lost,387907290.0,nAn,RCV000030763,N,Leber congenital amaurosis 9,"MONDO:MONDO:0012056, <a target=_blank href='ht...",germline,GRCh38,1p36.22,"[[""g.9982699T>C"", ""-""], [""g.44272T>C"", ""-""], [...",nAn,1,"Sep 01,2012",nAn,<a target=_blank href='http://www.omim.org/ent...,"NM_022787.4(NMNAT1):c.838T>C (p.Ter280Gln), Ty...",P,GERM,SUBST,1,0,37132,nAn,1,9982699.0,37132.0,T,C,.,.,"ALLELEID=45793;CLNDISDB=MONDO:MONDO:0012056,Me...",chr1,Pathogenic,nAn,nAn,A,A,.,NM_022787.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,ACACTGCAGAAGCTAAGACATAGGAATTCTACAGCATGATA,838,ACACTGCAGAAGCTAAGACATAG,NaN,Ter,NaN,1,NMNAT1,NaN,NaN,TAG,TAG
411,chr1,11790681,11790682,T>C,0,-,11790681,11790682,"210,0,0",1,1,0,187906|NM_005957.5(MTHFR):c.1969T>C (p.Ter657Arg),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,4524|MTHFR,stop lost,768434408.0,nAn,RCV000167626,N,Homocystinuria due to methylene tetrahydrofola...,"MONDO:MONDO:0009353, <a target=_blank href='ht...",germline,GRCh38,1p36.22,"[[""-"", ""-""], [""-"", ""p.Ter657Arg""], [""g.1179068...",nAn,1,nAn,nAn,ClinGen:CA198664,"NM_005957.5(MTHFR):c.1969T>C (p.Ter657Arg), Ty...",P,GERM,SUBST,1,0,187906,nAn,1,11790682.0,187906.0,A,G,.,.,AF_EXAC=0.00001;ALLELEID=185757;CLNDISDB=MONDO...,chr1,Pathogenic,nAn,nAn,A,G,.,NM_005957.5,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,CGAGAGAAACGGAGGCTCCATGACCCTGCGTCCTGACGCCC,1969,CGAGAGAAACGGAGGCTCCATGA,NaN,Ter,NaN,0,MTHFR,NaN,NaN,TGA,TGA
1824,chr1,43930200,43930201,G>T,0,+,43930200,43930201,"210,0,0",1,1,0,30588|NM_006279.5(ST3GAL3):c.1108G>T (p.Asp370...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,6487|ST3GAL3,stop lost,1557612719.0,nAn,RCV000023552,N,"Mental retardation,autosomal recessive 12","MONDO:MONDO:0012612, <a target=_blank href='ht...",germline,GRCh38,1p34.1,"[[""g.43930201G>T"", ""-""], [""g.227656G>T"", ""-""],...",nAn,1,"Sep 09,2011",nAn,<a target=_blank href='http://www.omim.org/ent...,"NM_006279.5(ST3GAL3):c.1108G>T (p.Asp370Tyr), ...",P,GERM,SUBST,1,0,30588,nAn,1,43930201.0,30588.0,G,T,.,.,"ALLELEID=39545;CLNDISDB=MONDO:MONDO:0012612,Me...",chr1,Pathogenic,nAn,nAn,T,A,.,NM_006279.5,43930132.0,43930218.0,+,0.0,NM_006279.5,17.0,69.0,0.0,TGAAAGCTCGCGTCATCACTGATCTAAGCAGTGGCATCTGA,1108,TGAAAGCTCGCGTCATCACTGATCTAAGCAGTGGCATCTGA,GAT,Asp,Asp,0,ST3GAL3,0.0,NaN,NaN,GAT
2206,chr1,47417272,47417273,T>C,0,+,47417272,47417273,"210,0,0",1,1,0,1077110|NM_012186.3(FOXE3):c.958T>C (p.Ter320Arg),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2301|FOXE3,stop lost,nAn,nAn,RCV001391241,N,"Cataract 34,multiple types","MONDO:MONDO:0013067, <a target=_blank href='ht...",germline,GRCh38,1p33,"[[""g.6202T>C"", ""-""], [""c.958T>C"", ""p.Ter320Arg...",nAn,1,"May 01,2021",nAn,nAn,"NM_012186.3(FOXE3):c.958T>C (p.Ter320Arg), Typ...",P,GERM,SUBST,1,0,1077110,nAn,1,47417273.0,1077110.0,T,C,.,.,"ALLELEID=1065749;CLNDISDB=MONDO:MOND

In [21]:
# # to check the cases in which we did not receive a value in the codon of the stop lost
# new_final_df[(new_final_df['molConseq']=='stop lost')& new_final_df['codon_seq'].isna()]

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf_x,chromEnd_gtf_x,strand_gtf_x,loc_in_frame,transcript_id_x,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,new_codon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif,codon,codon_seq
31351,chr16,173597,173598,T>G,2,+,173597,173598,"210,0,0",1,1,0,15653|NM_000517.4(HBA2):c.427T>G (p.Ter143Glu),Pathogenic/Likely pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016939|RCV000759054|RCV001285922|RCV0012...,N,HEMOGLOBIN SEAL ROCK|not provided|none provide...,|MedGen:CN517202|MedGen:CN235283|MONDO:MONDO:0...,germline,GRCh38,16p13.3,"[[""g.1948T>G"", ""-""], [""-"", ""-""], [""-"", ""-""], [...",nAn,5,"Mar 23,2020",nAn,"HBVAR:706, ClinGen:CA125597, <a target=_blank ...","NM_000517.4(HBA2):c.427T>G (p.Ter143Glu), Type...",LP,GERM,SUBST,1,2,15653,nAn,16,173598.0,15653.0,T,G,.,.,"ALLELEID=30692;CLNDISDB=.|MONDO:MONDO:0011399,...",chr16,Pathogenic/Likely_pathogenic,nAn,nAn,T,A,.,NM_000517.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,TGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTAGCC,427,NaN,NaN,Ter,NaN,1,HBA2,NaN,NaN,NaN,NaN
31352,chr16,173597,173598,T>A,1,+,173597,173598,"210,0,0",1,1,0,15626|NM_000517.4(HBA2):c.427T>A (p.Ter143Lys),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016893|RCV000022603|RCV000985726,N,"HEMOGLOBIN ICARIA|Hemoglobin H disease,nondele...",|MedGen:C3279561|MedGen:CN517202,germline,GRCh38,16p13.3,"[[""g.1948T>A"", ""-""], [""-"", ""-""], [""g.173598T>A...",nAn,2,"Jun 25,2019",nAn,"HBVAR:704, ClinGen:CA125550, <a target=_blank ...","NM_000517.4(HBA2):c.427T>A (p.Ter143Lys), Type...",P,GERM,SUBST,1,1,15626,nAn,16,173598.0,15626.0,T,A,.,.,ALLELEID=30665;CLNDISDB=.|MedGen:C3279561|MedG...,chr16,Pathogenic,nAn,nAn,T,A,.,NM_000517.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,TGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTAGCC,427,NaN,NaN,Ter,NaN,1,HBA2,NaN,NaN,NaN,NaN
31353,chr16,173597,173598,T>C,2,+,173597,173598,"210,0,0",1,1,0,15624|NM_000517.4(HBA2):c.427T>C (p.Ter143Gln),Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016891|RCV000022602|RCV000169546|RCV0005...,Y,Hemoglobin constant spring|Hemoglobin H diseas...,MedGen:C3891114|MedGen:C3279561|MONDO:MONDO:00...,"germline,inherited,unknown",GRCh38,16p13.3,"[[""g.1948T>C"", ""-""], [""-"", ""-""], [""c.427T>C"", ...",nAn,9,"Oct 15,2020",nAn,"HBVAR:703, ClinGen:CA125546, <a target=_blank ...","NM_000517.4(HBA2):c.427T>C (p.Ter143Gln), Type...",P,GERM,SUBST,1,2,15624,nAn,16,173598.0,15624.0,T,C,.,.,AF_EXAC=0.00005;AF_TGP=0.00020;ALLELEID=30663;...,chr16,Pathogenic,nAn,nAn,T,A,.,NM_000517.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,TGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTAGCC,427,NaN,NaN,Ter,NaN,1,HBA2,NaN,NaN,NaN,NaN
31355,chr16,173599,173600,A>T,2,+,173599,173600,"210,0,0",1,1,0,15652|NM_000517.4(HBA2):c.429A>T (p.Ter143Tyr),Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41412046.0,nAn,RCV000016938|RCV000508511|RCV001282879,N,"Hemoglobin H disease,nondeletional|not provide...",MedGen:C3279561|MedGen:CN517202|MedGen:CN235283,germline,GRCh38,16p13.3,"[[""-"", ""-""], [""g.1950A>T"", ""-""], [""g.173600A>T...",nAn,3,"Aug 24,2020",nAn,"HBVAR:707, ClinGen:CA125595,

In [22]:
# # Throwing out previous columns that are no longer relevant
# new_final_df = new_final_df.drop(['codon','new_codon_seq'],axis=1)
# new_final_df

,chrom,chromStart,chromEnd,mutation,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf_x,chromEnd_gtf_x,strand_gtf_x,loc_in_frame,transcript_id_x,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc,dna_seq,location_on_exon,new_exon_seq,aa_in_origin_name,AA,DNA off-target hits -minIdentity=85,Gene,RNA off-target hits -minIdentity=85,ADAR motif,codon_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,0.0,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,Arg,Arg,0,SAMD11,0.0,yes,CGA
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,1.0,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,Ala,Ala,0,KLHL17,0.0,NaN,GCG
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,NaN,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,0,ISG15,NaN,yes,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,0.0,ACC

In [23]:
# # save df
# new_final_df.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_merged_ver_2_with_DNA_RNA_seq_and_codon_and_off_target_and_motif_22_03_23_after_codon_fix_stop_lost_fix.tsv", index=False,sep="\t")